## Events
### Inherit from AbstractEvent

In [1]:
import os

os.chdir("..")

In [2]:
from pxtrade.events import (
    AbstractEvent,
    AssetPriceEvent,
    FxRateEvent,
    IndicatorEvent,
    TradeEvent,
)


All events in pxtrade inherit from AbstractEvent.

In [3]:
for event_class in [AssetPriceEvent, FxRateEvent, IndicatorEvent, TradeEvent]:
    print(issubclass(event_class, AbstractEvent))


True
True
True
True


### Can only be processed once
All events have a datetime and a processed (boolean) attribute. If an event is processed more than once it suggests something has gone wrong in our logic. Using an asset price event as an example.

In [4]:
from pxtrade.assets import reset, Stock

In [5]:
reset()
stock = Stock("ZZB AU", 2.50, currency_code="AUD")
print(stock)

Stock('ZZB AU', 2.5, currency_code='AUD')


In [6]:
from datetime import datetime

dt = datetime(2020, 10, 31, 12, 30)
event = AssetPriceEvent(stock, dt, 2.60)
print(event)

AssetPriceEvent(Stock('ZZB AU'), 2020-10-31 12:30:00, 2.6)


So our stock, 'ZZB AU', currently has a price of AUD 2.50. There is an unprocessed asset price event on 31 Oct 2020 where the price moves to 2.60.

In [7]:
stock.price

2.5

In [8]:
event.datetime, event.processed

(datetime.datetime(2020, 10, 31, 12, 30), False)

Once the event is processed the asset price moves and the event's processed attribute changes to True.

In [9]:
event.process()

In [10]:
print(stock)

Stock('ZZB AU', 2.6, currency_code='AUD')


In [11]:
event.processed

True

This event cannot be processed more than once.

In [12]:
try:
    event.process()
except Exception as e:
    print(e)

Event has already been processed.


### Event Examples

In [13]:
from datetime import datetime
from pxtrade import Trade
from pxtrade.assets import reset, Cash, Stock, FxRate, Portfolio

reset()
aud = Cash("AUD")
usd = Cash("USD")
audusd = FxRate("AUDUSD", 0.65)
stock = Stock("ZZB", 2.50)
portfolio = Portfolio("USD")
dt = datetime(2020, 10, 31)

In [14]:
asset_event = AssetPriceEvent(stock, dt, 2.55)
print(asset_event)

AssetPriceEvent(Stock('ZZB'), 2020-10-31 00:00:00, 2.55)


In [15]:
fx_event = FxRateEvent(audusd, dt, 0.75)
print(fx_event)

FxRateEvent('AUDUSD', 2020-10-31 00:00:00, 0.75)


In [16]:
indicator_event = IndicatorEvent("VIX", dt, 25)
print(indicator_event)

IndicatorEvent('VIX', 2020-10-31 00:00:00, 25)


In [17]:
trade = Trade(portfolio, stock, 100)
trade_event = TradeEvent(dt, trade)
print(trade_event)

TradeEvent(2020-10-31 00:00:00, Trade(Portfolio('USD'), 'ZZB', 100))


And if we process these events.

In [18]:
print("Stock before: ", stock)
asset_event.process()
print("Stock after: ", stock)

Stock before:  Stock('ZZB', 2.5, currency_code='USD')
Stock after:  Stock('ZZB', 2.55, currency_code='USD')


In [19]:
print("AUDUSD before: ", audusd.rate)
fx_event.process()
print("AUDUSD after: ", audusd.rate)

AUDUSD before:  0.65
AUDUSD after:  0.75


Indicators are associated with some backtest, so we'll cover their processing later.

In [20]:
print("Portfolio before: ", portfolio)
trade_event.process()
print("*" * 10)
print("Portfolio after: ", portfolio)

Portfolio before:  Portfolio('USD')
**********
Portfolio after:  Portfolio('USD'):
Stock('ZZB', 2.55, currency_code='USD'): 100
Cash('USD', 1.0, currency_code='USD'): -255


### Event Queues
For a given backtest we are likely to be queueing multiple events of different types. These events are queued in time priority with the oldest events being processed first.

In [21]:
from pxtrade import EventsQueue

reset()
queue = EventsQueue()
stock = Stock("ZZZ")
event1 = AssetPriceEvent(stock, datetime(2020, 9, 3), 2.65)
event2 = AssetPriceEvent(stock, datetime(2020, 9, 1), 2.55)
event3 = AssetPriceEvent(stock, datetime(2020, 9, 2), 2.60)
[queue.put(event) for event in [event1, event2, event3]]

while len(queue) > 0:
    datetime, event = queue.get()
    print(event)


AssetPriceEvent(Stock('ZZZ'), 2020-09-01 00:00:00, 2.55)
AssetPriceEvent(Stock('ZZZ'), 2020-09-02 00:00:00, 2.6)
AssetPriceEvent(Stock('ZZZ'), 2020-09-03 00:00:00, 2.65)


Note that the event with the oldest datetime is the first to be processed.

Whilst it's useful to know how the event queue works it'll largely run in the background and you won't have to interact directly with it.

***